In [1]:
import BCI2000Tools
from   BCI2000Tools import Chain
from   BCI2000Tools import Remote
import numpy as np

import time
from   scipy import signal

In [2]:
# open BCI2000 and run the pipeline with ADM encoding
BCI2000 = Remote.BCI2000Remote()
BCI2000.Connect()
BCI2000.Execute('show window')

# real-time stream from Micromed SystemPlus
#BCI2000.StartupModules(['Micromed', 'ADM', 'DummyApplication'])

# real-time stream from pre-recorded data
BCI2000.StartupModules(['FilePlayback', 'ADM', 'DummyApplication'])

1

In [3]:
BCI2000.GetStates()["Refresh"]

0

In [4]:
# define sampling frequency
fs        = 2000
#define number of channels
nch       = 23
startCh   = 1
stopCh    = startCh+nch
chParams  = [str(c) for c in np.arange(startCh,stopCh)]
repParams = 2

for i in range(len(chParams)):
    chParams[i] = 'Ch'+chParams[i]

initialRoot = BCI2000Tools.bci2000root()

In [5]:
bipolar=False  #set to True to activate bipolar montage

In [6]:
BCI2000.LoadParametersRemote('..\\tools\\python\\demo.prm')

1

In [7]:
# set BCI2000 parameters
BCI2000.Parameters['SourceCh'].Value         = str(nch)
BCI2000.Parameters['SamplingRate'].Value     = str(fs)+'Hz'
BCI2000.Parameters['SampleBlockSize'].Value  = '64'

# Micromed parameters
#BCI2000.Parameters['ConditionMask'].Value    = '0'
#BCI2000.Parameters['SubjectRun'].Value       = '1'

# FilePlayback parameters
datfile     = BCI2000Tools.bci2000path('derivatives/sub-05/ses-SITUATION1A/sub-05_ses-SITUATION1A_task-BCI.dat')
BCI2000.Parameters['PlaybackFileName'].Value = datfile

# visualization 
BCI2000.Parameters['VisualizeTiming'].Value  = '0'
BCI2000.Parameters['VisualizeSource'].Value    = '1'
BCI2000.Parameters['VisualizeFIRFilter'].Value = '1'
BCI2000.Parameters['VisualizeTransmissionFilter'].Value = '0'
BCI2000.Parameters['VisualizeADMFilter'].Value = '1'

# set ADM output files
BCI2000.Parameters['OutputA'].Value = '..//UPDN_demo.txt'
BCI2000.Parameters['OutputB'].Value = '..//UPDN_demo.txt'

# transmit all channels for filtering and ADM conversion
BCI2000.Parameters['TransmitChList'].Value = chParams
if(bipolar):
    chList1 = chParams[1:]
    chList2 = chParams[:-1]
    BCI2000.Parameters['ChList1'].Value = chList1*repParams
    BCI2000.Parameters['ChList2'].Value = chList2*repParams
else:
    chList1 = chParams
    chList2 = chParams
    BCI2000.Parameters['ChList1'].Value = chParams*repParams
    BCI2000.Parameters['ChList2'].Value = chParams*repParams

# define channel names for ADM conversion 
nch  = len(chList1)*repParams
admCh = []
for i in range(nch):
    admCh.append(str(i))

# ADM parameters
admLenW =  ["0.05"]*int(nch//repParams) # EEG
admLenW += ["0.005"]*int(nch//repParams)# HFO
BCI2000.Parameters['LenWindow'].Value = admLenW

admPerc =  ["40"]*int(nch//repParams) # EEG
admPerc += ["50"]*int(nch//repParams) # HFO
BCI2000.Parameters['percentileDelta'].Value    = admPerc
BCI2000.Parameters['RefractoryPeriod'].Value   = str(1/fs)
BCI2000.Parameters['SelectedChannel'].Value    = admCh
BCI2000.Parameters['LenWindow'].Value          = admLenW
BCI2000.Parameters['LengthTuning'].Value       = '5'

# FIR filter parameters for HFO band (250-500)Hz
gainHFO = 100 #amplitude gain, just for visualization purposes
f1      = 250 
f2      = 500
numtaps = 64
FIR_HFO = np.array([signal.firwin(numtaps, [f1, f2], pass_zero=False,fs=fs)])*gainHFO
FIR_HFO = np.repeat(FIR_HFO,int(nch//repParams),axis=0)
print(FIR_HFO.shape)

# FIR filter parameters for EEG band (4-80)Hz
f1 = 4
f2 = 80
FIR_EEG = np.array([signal.firwin(numtaps, [f1, f2], pass_zero=False,fs=fs)])
FIR_EEG = np.repeat(FIR_EEG,int(nch//repParams),axis=0)
print(FIR_EEG.shape)

# save filter parameters
matFilter = np.concatenate((FIR_EEG,FIR_HFO),axis=0)
print(matFilter.shape)
np.savetxt('matFilt.txt',matFilter,delimiter='\t')

# load filter paramters into BCI2000
BCI2000.Parameters['FIRCoefficients'].Value = np.loadtxt('matFilt.txt')
BCI2000.Parameters['FIRIntegration'].Value = '0'

(23, 64)
(23, 64)
(46, 64)


In [8]:
# FilePlayback
LRec       = 840000/fs #sec

# run BCI2000
BCI2000.SetConfig()
BCI2000.Start()
running = True

tstart     = time.time()
while(running):
    time.sleep(10)
    if(time.time()-tstart>=LRec):
        running=False
        
# close BCI2000
BCI2000.Stop()
BCI2000.Quit()
BCI2000.Disconnect()

1